## Coding Assignment 3
### Gaurav Shah (gns274)

In [245]:
#Import Libraries
import numpy as np
import pandas as pd
from scipy import integrate, optimize
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import shift
from scipy.stats import norm

In [269]:
#Define Variables (1)
a = 0.03
b = 0.002
s = 0.005
sigma_r = 0.02
c = 0.4
kappa_2 = 0.05
rho_infinity = 0.4
T = 10
notional = 50
lambda_A = 0.04
lambda_B = 0.02
recovery_rate = 0.4
spread_B = 0.012
alpha_ead = 1.4
probability_default = 0.005
loss_given_default = 0.6
rho_s = 0.4
rho_l = 0.2

In [270]:
#Define Variables (2)
v = np.sqrt(np.square(1/c) - 1 - 2*((rho_infinity/c) - 1))
sigma_1 = c * sigma_r
sigma_2 = v * sigma_1
rho_x = ((rho_infinity/c) - 1)/v
kappa = np.matrix([[0,0],
                  [0,kappa_2]])
sigma_x = np.matrix([[sigma_1,0],
                  [0,sigma_2]])
rho_Z_1 = rho_l
rho_Z_2 = rho_s * ((v**2 + 1 + 2*rho_x*v) ** 0.5) - rho_Z_1*v 

In [271]:
#Define Variables (3)
k_l = 0.03
sigma_l = 0.01

In [272]:
#Function for calculating y(t)
def matrix_calc(t_iter):
    B_t = np.matrix([[1,0],
                    [0,np.exp(-kappa_2*t_iter)]])
    q_t = np.matrix([[sigma_1,rho_x*sigma_1*sigma_2*(np.exp(kappa_2*t_iter))],
                    [rho_x*sigma_1*sigma_2*(np.exp(kappa_2*t_iter)),sigma_2*(np.exp(kappa_2*t_iter))]])
    temp_t = q_t.T * q_t
    integrate_temp_t = np.matrix([[np.square(sigma_1)*t_iter + (np.square(rho_x*sigma_1*sigma_2)*(np.exp(2*kappa_2*t_iter)-1))/(2*kappa_2), (np.square(sigma_1)*rho_x*sigma_2*(np.exp(kappa_2*t_iter)-1))/(kappa_2) + (np.square(sigma_2)*rho_x*sigma_1*(np.exp(2*kappa_2*t_iter)-1))/(2*kappa_2)],
                                   [(np.square(sigma_1)*rho_x*sigma_2*(np.exp(kappa_2*t_iter)-1))/(kappa_2) + (np.square(sigma_2)*rho_x*sigma_1*(np.exp(2*kappa_2*t_iter)-1))/(2*kappa_2), (np.square(rho_x*sigma_1*sigma_2)*(np.exp(2*kappa_2*t_iter)-1))/(2*kappa_2) + (np.square(sigma_2)*(np.exp(2*kappa_2*t_iter)-1))/(2*kappa_2)]])
    y_t = B_t * integrate_temp_t * B_t
    return y_t

In [273]:
#Function for calculating F(t,x(t),T)
def F(t_iter,x_t,T_iter,y_t,P_0_t):
    G_t_T = np.matrix([[T_iter-t_iter],
               [(1 - np.exp(-kappa_2*(T_iter-t_iter)))/kappa_2]])
    P_0_T = np.exp(-a*T_iter-(b*(T_iter**2)/2))
    P_t_T = (P_0_T/P_0_t) * np.exp(-G_t_T.T * x_t - (G_t_T.T * y_t * G_t_T)/2)
    return P_t_T

In [274]:
#Function for calculating the values of Payer and Receiver legs
#Returns the payer and receciver legs of any swap
def payer_receiver(t_iter,x_t,y_t,P_0_t,delta_T, hpar_value_func):
    sum_payer = 0
    sum_receiver = 0
    for T_iter in np.arange((int(t_iter/0.5)+1)*0.5,10.0001,delta_T):
        if(T_iter==(int(t_iter/0.5)+1)*0.5):
            sum_receiver = sum_receiver + notional*np.exp(s*delta_T)*F(t_iter,x_t,T_iter-delta_T,y_t,P_0_t)
            temp = F(t_iter,x_t,T_iter,y_t,P_0_t)
            sum_payer = sum_payer + notional*temp*(1+(hpar_value_func*delta_T))
        else:
            sum_receiver = sum_receiver + notional*np.exp(s*delta_T)*temp
            temp = F(t_iter,x_t,T_iter,y_t,P_0_t)
            sum_payer = sum_payer + notional*temp*(1+(hpar_value_func*delta_T))
    return (sum_payer,sum_receiver)
#V(t,x) = sum_receiver - sum_payer

In [275]:
#Calculating all the y(t) to avoid repeating y(t) calculations as y(t) is only dependent on time
delta_t=1/52
matrix_calc_list=[]
for i in range(0,int(T/delta_t)+1):
    t_iter=i*delta_t
    matrix_calc_list.append(matrix_calc(t_iter))

In [276]:
#Calculating all the y(t) to avoid repeating y(t) calculations as y(t) is only dependent on time
delta_t=1/52
y_t_l_list=[]
for i in range(0,int(T/delta_t)+1):
    t_iter=i*delta_t
    y_t_l_list.append(sigma_l**2/(2*k_l)*(1-np.exp(-2*k_l*i)))

In [277]:
#Function for Calculating Unilateral CVA
def calculate_CVA(total_payer_swap_array, default_int_counterparty):
    cva = 0
    for i in range(len(total_payer_swap_array)):
        if(i == 0):
            cva = cva + (total_payer_swap_array[i] * (np.exp(-default_int_counterparty*(i/52)) - 1))
        else:
            cva = cva + (total_payer_swap_array[i] * (np.exp(-default_int_counterparty*(i/52)) - np.exp(-default_int_counterparty*(i-1)/52)))
    return cva

Q.1

In [131]:
T_sim=1
delta_t = 1/52
delta_T = 0.5
u = np.matrix([[1],
              [1]])
n_paths = 1000
hpar_list=[0.02,0.04,0.06,0.08]
h_payer_EPE=[]
h_receiver_EPE=[]
h_payer_EEPE=[]
h_receiver_EEPE=[]
h_m_payer=[]
h_m_receiver=[]
EAD_payer=[]
EAD_receiver=[]

for hpar_value in hpar_list: # h_par loop for different values of fixed coupon
    total_payer_swap_array = np.zeros(521)
    total_receiver_swap_array = np.zeros(521)
    EE_star_payer=np.array([])
    EE_star_receiver=np.array([])
    total_pvee_payer_swap_array = np.zeros(521)
    total_pvee_receiver_swap_array = np.zeros(521)
    for j in range(n_paths): #n_path loop for monte carlo
        x_previous = np.matrix([[0],
                               [0]])
        payer_swap_array = np.array([])
        receiver_swap_array = np.array([])
        z_array = np.random.normal(0,1,2*(int(T/delta_t))+1)
        discount_factor_array = np.array([])
        for i in range(0,int(T/delta_t)+1): # i loop for creating of weekly sample paths 
            t_iter = i*delta_t
            z_1 = z_array[i]
            z_2 = z_array[i+int(T/delta_t)]
            z_3 = rho_x * z_1 + ((1 - (rho_x**2))**0.5) * z_2
            z_t = np.matrix([[z_1],
                        [z_3]])
#             y_t = matrix_calc(t_iter)
            y_t=matrix_calc_list[i]
            P_0_t = np.exp(-a*t_iter-(b*(t_iter**2)/2))
            payer, receiver = payer_receiver(t_iter,x_previous,y_t,P_0_t,delta_T,hpar_value)
            dx_t = (y_t*u - kappa*x_previous)*delta_t + sigma_x.T*z_t*(delta_t**0.5)
            discount_factor = np.exp(-(a*t_iter+(b*(t_iter**2)/2) + (x_previous[0,0] + x_previous[1,0])*t_iter))
            x_previous = x_previous + dx_t
            
            discount_factor_array = np.append(discount_factor_array,[discount_factor])
            
            payer_swap_array = np.append(payer_swap_array,[max(receiver-payer,0)]) # array storing weekly exposure values of payer swap
            receiver_swap_array = np.append(receiver_swap_array,[max(payer-receiver,0)]) # # array storing weekly exposure values of receiver swap
        
        total_payer_swap_array = total_payer_swap_array + payer_swap_array
        total_receiver_swap_array = total_receiver_swap_array + receiver_swap_array
        
        total_pvee_payer_swap_array = total_pvee_payer_swap_array + (payer_swap_array * discount_factor_array)
        total_pvee_receiver_swap_array = total_pvee_receiver_swap_array + (receiver_swap_array * discount_factor_array)
        
    total_payer_swap_array = total_payer_swap_array/n_paths # E(E(t)) array for payer swap 
    total_receiver_swap_array = total_receiver_swap_array/n_paths # E(E(t)) array for receiver swap
    
    total_pvee_payer_swap_array = total_pvee_payer_swap_array/n_paths
    total_pvee_receiver_swap_array = total_pvee_receiver_swap_array/n_paths
    
    unilateral_cva_payer = -(1-recovery_rate)*calculate_CVA(total_pvee_payer_swap_array, lambda_A) # calculate unilateral CVA with PVEE(t) and Counterparty default intensity
    unilateral_cva_receiver = -(1-recovery_rate)*calculate_CVA(total_pvee_receiver_swap_array, lambda_A) # calculate unilateral CVA with PVEE(t) and Counterparty default intensity
    
    h_payer_EPE.append(np.sum(total_payer_swap_array[0:53])*delta_t/T_sim) # formula as per notes and [0:53] indicates 1 year data
    h_receiver_EPE.append(np.sum(total_receiver_swap_array[0:53])*delta_t/T_sim) # formula as per notes and [0:53] indicates 1 year data
    EE_star_payer=np.maximum.accumulate(total_payer_swap_array[0:53]) # formula as per notes and [0:53] indicates 1 year data
    EE_star_receiver=np.maximum.accumulate(total_receiver_swap_array[0:53]) # formula as per notes and [0:53] indicates 1 year data
    h_payer_EEPE.append(np.sum(EE_star_payer)*delta_t/T_sim) # formula as per notes
    h_receiver_EEPE.append(np.sum(EE_star_receiver)*delta_t/T_sim) # formula as per notes
    m_payer = (np.sum(total_payer_swap_array)*delta_t)/(np.sum(EE_star_payer)*delta_t) # formula as per notes
    m_receiver = (np.sum(total_receiver_swap_array)*delta_t)/(np.sum(EE_star_receiver)*delta_t) # formula as per notes
    h_m_payer.append(max(1,min(5,m_payer))) # formula as per notes
    h_m_receiver.append(max(1,min(5,m_receiver))) # formula as per notes
    EAD_payer.append(1.06*max(alpha_ead*(np.sum(EE_star_payer)*delta_t/T_sim)-unilateral_cva_payer,0)) # formula as per notes
    EAD_receiver.append(1.06*max(alpha_ead*(np.sum(EE_star_receiver)*delta_t/T_sim)-unilateral_cva_receiver,0)) # formula as per notes
result_capital_payer=pd.DataFrame([h_payer_EPE,h_payer_EEPE,EAD_payer,h_m_payer],columns=['0.02','0.04','0.06','0.08'],index=['EPE(1) ($MM)','EEPE(1) ($MM)','EAD ($MM)','M (Years)'])
result_capital_receiver=pd.DataFrame([h_receiver_EPE,h_receiver_EEPE,EAD_receiver,h_m_receiver],columns=['0.02','0.04','0.06','0.08'],index=['EPE(1) ($MM)','EEPE(1) ($MM)','EAD ($MM)','M (Years)'])

In [132]:
print("Payer Swap")
print(result_capital_payer)
print("\nReceiver Swap")
print(result_capital_receiver)

Payer Swap
                    0.02      0.04      0.06      0.08
EPE(1) ($MM)   10.486655  3.354126  0.284911  0.008981
EEPE(1) ($MM)  10.631042  3.356567  0.284911  0.009145
EAD ($MM)      14.455795  4.310570  0.139207  0.000000
M (Years)       5.000000  5.000000  5.000000  5.000000

Receiver Swap
                   0.02      0.04      0.06       0.08
EPE(1) ($MM)   0.045363  0.963003  6.410738  14.649245
EEPE(1) ($MM)  0.046371  0.974122  6.673503  15.182165
EAD ($MM)      0.000000  1.105118  9.055398  20.855615
M (Years)      5.000000  5.000000  5.000000   4.952442


Q.2

In [133]:
def k_calc(m,pd):
    b_y=(0.11852-0.05478*np.log(pd))
    k_x_y=(1+(m-2.5)*b_y)/(1-1.5*b_y)
    return k_x_y

In [134]:
airb_payer = []
airb_receiver = []
k_payer = []
k_receiver = []
rho_p = 0.24 - 0.12 * (1 - np.exp(-50 * probability_default)) # formula as per notes
temp_airb = norm.cdf((norm.ppf(probability_default) - np.sqrt(rho_p) * norm.ppf(0.001))/(np.sqrt(1 - rho_p))) - probability_default # formula as per notes
for i in range(4):
    k_payer.append(k_calc(h_m_payer[i],probability_default))
    k_receiver.append(k_calc(h_m_receiver[i],probability_default))
    payer_temp = EAD_payer[i]*loss_given_default*(temp_airb)*k_calc(h_m_payer[i],probability_default) # formula as per notes
    receiver_temp = EAD_receiver[i]*loss_given_default*(temp_airb)*k_calc(h_m_receiver[i],probability_default) # formula as per notes
    airb_payer.append(payer_temp)
    airb_receiver.append(receiver_temp)
airb_capital=pd.DataFrame([airb_payer,k_payer,airb_receiver,k_receiver],columns=['0.02','0.04','0.06','0.08'],index=['AIRB_Payer ($MM)','k_Payer','AIRB_Receiver ($MM)','k_Receiver'])

In [135]:
print(airb_capital)

                         0.02      0.04      0.06      0.08
AIRB_Payer ($MM)     4.203971  1.253581  0.040484  0.000000
k_Payer              5.226487  5.226487  5.226487  5.226487
AIRB_Receiver ($MM)  0.000000  0.321386  2.633451  6.006826
k_Receiver           5.226487  5.226487  5.226487  5.176237


Q.3

In [136]:
T_sim=1
delta_t = 1/52
delta_T = 0.5
u = np.matrix([[1],
              [1]])
n_paths = 1000
hpar_list=[0.02,0.04,0.06,0.08]
h_payer_EPE=[]
h_receiver_EPE=[]
h_payer_EEPE=[]
h_receiver_EEPE=[]
h_m_payer=[]
h_m_receiver=[]
EAD_payer=[]
EAD_receiver=[]

for hpar_value in hpar_list: # h_par loop for different values of fixed coupon
    total_payer_swap_array = np.zeros(521)
    total_receiver_swap_array = np.zeros(521)
    EE_star_payer=np.array([])
    EE_star_receiver=np.array([])
    total_pvee_payer_swap_array = np.zeros(521)
    total_pvee_receiver_swap_array = np.zeros(521)
    for j in range(n_paths): #n_path loop for monte carlo
        x_previous = np.matrix([[0],
                               [0]])
        payer_swap_array = np.array([])
        receiver_swap_array = np.array([])
        z_array = np.random.normal(0,1,2*(int(T/delta_t))+1)
        discount_factor_array = np.array([])
        for i in range(0,int(T/delta_t)+1): # i loop for creating of weekly sample paths 
            t_iter = i*delta_t
            z_1 = z_array[i]
            z_2 = z_array[i+int(T/delta_t)]
            z_3 = rho_x * z_1 + ((1 - (rho_x**2))**0.5) * z_2
            z_t = np.matrix([[z_1],
                        [z_3]])
#             y_t = matrix_calc(t_iter)
            y_t=matrix_calc_list[i]
            P_0_t = np.exp(-a*t_iter-(b*(t_iter**2)/2))
            payer, receiver = payer_receiver(t_iter,x_previous,y_t,P_0_t,delta_T,hpar_value)
            dx_t = (y_t*u - kappa*x_previous)*delta_t + sigma_x.T*z_t*(delta_t**0.5)
            
            discount_factor = np.exp(-(a*t_iter+(b*(t_iter**2)/2) + (x_previous[0,0] + x_previous[1,0])*t_iter))
            discount_factor_array = np.append(discount_factor_array,[discount_factor])
            
            x_previous = x_previous + dx_t
            #payer_swap_array = np.append(payer_swap_array,[max(receiver-payer,0)]) # array storing weekly exposure values of payer swap
            #receiver_swap_array = np.append(receiver_swap_array,[max(payer-receiver,0)]) # # array storing weekly exposure values of receiver swap
            payer_swap_array = np.append(payer_swap_array,[receiver-payer])
            receiver_swap_array= np.append(receiver_swap_array,[payer-receiver])
        payer_swap_array = payer_swap_array - shift(payer_swap_array, 2) # V(t) - V(t-2) for payer swap
        payer_swap_array[payer_swap_array < 0] = 0
        receiver_swap_array = receiver_swap_array - shift(receiver_swap_array, 2) # V(t) - V(t-2) for receiver swap
        receiver_swap_array[receiver_swap_array < 0] = 0
        
        total_payer_swap_array = total_payer_swap_array + payer_swap_array
        total_receiver_swap_array = total_receiver_swap_array + receiver_swap_array
        
        total_pvee_payer_swap_array = total_pvee_payer_swap_array + (payer_swap_array * discount_factor_array)
        total_pvee_receiver_swap_array = total_pvee_receiver_swap_array + (receiver_swap_array * discount_factor_array)
        
    total_payer_swap_array = total_payer_swap_array/n_paths # E(E(t)) array for payer swap 
    total_receiver_swap_array = total_receiver_swap_array/n_paths # E(E(t)) array for receiver swap
    
    total_pvee_payer_swap_array = total_pvee_payer_swap_array/n_paths
    total_pvee_receiver_swap_array = total_pvee_receiver_swap_array/n_paths
    
    unilateral_cva_payer = -(1-recovery_rate)*calculate_CVA(total_pvee_payer_swap_array, lambda_A) # calculate unilateral CVA with PVEE(t) and Counterparty default intensity
    unilateral_cva_receiver = -(1-recovery_rate)*calculate_CVA(total_pvee_receiver_swap_array, lambda_A) # calculate unilateral CVA with PVEE(t) and Counterparty default intensity

    h_payer_EPE.append(np.sum(total_payer_swap_array[0:53])*delta_t/T_sim)
    h_receiver_EPE.append(np.sum(total_receiver_swap_array[0:53])*delta_t/T_sim)
    EE_star_payer=np.maximum.accumulate(total_payer_swap_array[0:53])
    EE_star_receiver=np.maximum.accumulate(total_receiver_swap_array[0:53])
    h_payer_EEPE.append(np.sum(EE_star_payer)*delta_t/T_sim)
    h_receiver_EEPE.append(np.sum(EE_star_receiver)*delta_t/T_sim)
    m_payer = (np.sum(total_payer_swap_array)*delta_t)/(np.sum(EE_star_payer)*delta_t)
    m_receiver = (np.sum(total_receiver_swap_array)*delta_t)/(np.sum(EE_star_receiver)*delta_t)
    h_m_payer.append(max(1,min(5,m_payer)))
    h_m_receiver.append(max(1,min(5,m_receiver)))
    EAD_payer.append(1.06*max(alpha_ead*(np.sum(EE_star_payer)*delta_t/T_sim)-unilateral_cva_payer,0))
    EAD_receiver.append(1.06*max(alpha_ead*(np.sum(EE_star_receiver)*delta_t/T_sim)-unilateral_cva_receiver,0))
result_capital_payer=pd.DataFrame([h_payer_EPE,h_payer_EEPE,EAD_payer,h_m_payer],columns=['0.02','0.04','0.06','0.08'],index=['EPE(1) ($MM)','EEPE(1) ($MM)','EAD ($MM)','M (Years)'])
result_capital_receiver=pd.DataFrame([h_receiver_EPE,h_receiver_EEPE,EAD_receiver,h_m_receiver],columns=['0.02','0.04','0.06','0.08'],index=['EPE(1) ($MM)','EEPE(1) ($MM)','EAD ($MM)','M (Years)'])

In [137]:
print("Payer Swap")
print(result_capital_payer)
print("\nReceiver Swap")
print(result_capital_receiver)

Payer Swap
                    0.02      0.04      0.06      0.08
EPE(1) ($MM)    0.861481  0.605757  0.590102  0.659836
EEPE(1) ($MM)  10.552206  2.045142  0.788851  1.008949
EAD ($MM)      15.595959  2.968600  1.098362  1.416342
M (Years)       1.000000  1.648051  4.595667  4.004862

Receiver Swap
                   0.02      0.04      0.06       0.08
EPE(1) ($MM)   0.481061  0.522500  0.805367   1.168600
EEPE(1) ($MM)  0.601182  0.590025  6.428363  14.911291
EAD ($MM)      0.828427  0.810473  9.469774  22.051323
M (Years)      5.000000  5.000000  1.000000   1.000000


In [138]:
airb_payer = []
airb_receiver = []
k_payer = []
k_receiver = []
rho_p = 0.24 - 0.12 * (1 - np.exp(-50 * probability_default)) # formula as per notes
temp_airb = norm.cdf((norm.ppf(probability_default) - np.sqrt(rho_p) * norm.ppf(0.001))/(np.sqrt(1 - rho_p))) - probability_default # formula as per notes
for i in range(4):
    k_payer.append(k_calc(h_m_payer[i],probability_default))
    k_receiver.append(k_calc(h_m_receiver[i],probability_default))
    payer_temp = EAD_payer[i]*loss_given_default*(temp_airb)*k_calc(h_m_payer[i],probability_default)
    receiver_temp = EAD_receiver[i]*loss_given_default*(temp_airb)*k_calc(h_m_receiver[i],probability_default)
    airb_payer.append(payer_temp)
    airb_receiver.append(receiver_temp)
airb_capital=pd.DataFrame([airb_payer,k_payer,airb_receiver,k_receiver],columns=['0.02','0.04','0.06','0.08'],index=['AIRB_Payer ($MM)','k_Payer','AIRB_Receiver ($MM)','k_Receiver'])

In [139]:
print(airb_capital)

                         0.02      0.04      0.06      0.08
AIRB_Payer ($MM)     0.867801  0.278287  0.293310  0.329028
k_Payer              1.000000  1.684745  4.799260  4.175003
AIRB_Receiver ($MM)  0.240919  0.235698  0.526923  1.226994
k_Receiver           5.226487  5.226487  1.000000  1.000000


Q.4

In [196]:
T_sim=1
delta_t = 1/52
delta_T = 0.5
u = np.matrix([[1],
              [1]])
n_paths = 1000
hpar_list=[0.02,0.04,0.06,0.08]
h_payer_EPE=[]
h_receiver_EPE=[]
h_payer_EEPE=[]
h_receiver_EEPE=[]
h_m_payer=[]
h_m_receiver=[]
EAD_payer=[]
EAD_receiver=[]

for hpar_value in hpar_list: # h_par loop for different values of fixed coupon
    total_payer_swap_array = np.zeros(521)
    total_receiver_swap_array = np.zeros(521)
    EE_star_payer=np.array([])
    EE_star_receiver=np.array([])
    total_pvee_payer_swap_array = np.zeros(521)
    total_pvee_receiver_swap_array = np.zeros(521)
    for j in range(n_paths): #n_path loop for monte carlo
        x_previous = np.matrix([[0],
                               [0]])
        payer_swap_array = np.array([])
        receiver_swap_array = np.array([])
        z_array = np.random.normal(0,1,2*(int(T/delta_t))+1)
        discount_factor_array = np.array([])
        for i in range(0,int(T/delta_t)+1): # i loop for creating of weekly sample paths 
            t_iter = i*delta_t
            z_1 = z_array[i]
            z_2 = z_array[i+int(T/delta_t)]
            z_3 = rho_x * z_1 + ((1 - (rho_x**2))**0.5) * z_2
            z_t = np.matrix([[z_1],
                        [z_3]])
#             y_t = matrix_calc(t_iter)
            y_t=matrix_calc_list[i]
            P_0_t = np.exp(-a*t_iter-(b*(t_iter**2)/2))
            payer, receiver = payer_receiver(t_iter,x_previous,y_t,P_0_t,delta_T,hpar_value)
            dx_t = (y_t*u - kappa*x_previous)*delta_t + sigma_x.T*z_t*(delta_t**0.5)
            
            discount_factor = np.exp(-(a*t_iter+(b*(t_iter**2)/2) + (x_previous[0,0] + x_previous[1,0])*t_iter))
            discount_factor_array = np.append(discount_factor_array,[discount_factor])
            
            x_previous = x_previous + dx_t
            #payer_swap_array = np.append(payer_swap_array,[max(receiver-payer,0)]) # array storing weekly exposure values of payer swap
            #receiver_swap_array = np.append(receiver_swap_array,[max(payer-receiver,0)]) # # array storing weekly exposure values of receiver swap
            payer_swap_array = np.append(payer_swap_array,[receiver-payer])
            receiver_swap_array= np.append(receiver_swap_array,[payer-receiver])
        payer_swap_array = payer_swap_array - shift(payer_swap_array, 2) # V(t) - V(t-2) for payer swap
        payer_swap_array[payer_swap_array < 0] = 0
        receiver_swap_array = receiver_swap_array - shift(receiver_swap_array, 2) # V(t) - V(t-2) for receiver swap
        receiver_swap_array[receiver_swap_array < 0] = 0
        
        total_payer_swap_array = total_payer_swap_array + payer_swap_array
        total_receiver_swap_array = total_receiver_swap_array + receiver_swap_array
        
        total_pvee_payer_swap_array = total_pvee_payer_swap_array + (payer_swap_array * discount_factor_array)
        total_pvee_receiver_swap_array = total_pvee_receiver_swap_array + (receiver_swap_array * discount_factor_array)
        
    total_payer_swap_array = total_payer_swap_array/n_paths # E(E(t)) array for payer swap 
    total_receiver_swap_array = total_receiver_swap_array/n_paths # E(E(t)) array for receiver swap
    
    total_pvee_payer_swap_array = total_pvee_payer_swap_array/n_paths
    total_pvee_receiver_swap_array = total_pvee_receiver_swap_array/n_paths
    
    unilateral_cva_payer = -(1-recovery_rate)*calculate_CVA(total_pvee_payer_swap_array, lambda_A) # calculate unilateral CVA with PVEE(t) and Counterparty default intensity
    unilateral_cva_receiver = -(1-recovery_rate)*calculate_CVA(total_pvee_receiver_swap_array, lambda_A) # calculate unilateral CVA with PVEE(t) and Counterparty default intensity

    h_payer_EPE.append(np.sum(total_payer_swap_array[0:53])*delta_t/T_sim)
    h_receiver_EPE.append(np.sum(total_receiver_swap_array[0:53])*delta_t/T_sim)
    EE_star_payer=np.maximum.accumulate(total_payer_swap_array[2:53])
    EE_star_receiver=np.maximum.accumulate(total_receiver_swap_array[2:53])
    h_payer_EEPE.append(np.sum(EE_star_payer)*delta_t/T_sim)
    h_receiver_EEPE.append(np.sum(EE_star_receiver)*delta_t/T_sim)
    m_payer = (np.sum(total_payer_swap_array)*delta_t)/(np.sum(EE_star_payer)*delta_t)
    m_receiver = (np.sum(total_receiver_swap_array)*delta_t)/(np.sum(EE_star_receiver)*delta_t)
    h_m_payer.append(max(1,min(5,m_payer)))
    h_m_receiver.append(max(1,min(5,m_receiver)))
    EAD_payer.append(1.06*max(alpha_ead*(np.sum(EE_star_payer)*delta_t/T_sim)-unilateral_cva_payer,0))
    EAD_receiver.append(1.06*max(alpha_ead*(np.sum(EE_star_receiver)*delta_t/T_sim)-unilateral_cva_receiver,0))
result_capital_payer=pd.DataFrame([h_payer_EPE,h_payer_EEPE,EAD_payer,h_m_payer],columns=['0.02','0.04','0.06','0.08'],index=['EPE(1) ($MM)','EEPE(1) ($MM)','EAD ($MM)','M (Years)'])
result_capital_receiver=pd.DataFrame([h_receiver_EPE,h_receiver_EEPE,EAD_receiver,h_m_receiver],columns=['0.02','0.04','0.06','0.08'],index=['EPE(1) ($MM)','EEPE(1) ($MM)','EAD ($MM)','M (Years)'])

In [197]:
print("Payer Swap")
print(result_capital_payer)
print("\nReceiver Swap")
print(result_capital_receiver)

Payer Swap
                   0.02      0.04      0.06      0.08
EPE(1) ($MM)   0.862361  0.602546  0.591098  0.652888
EEPE(1) ($MM)  0.509816  0.589430  0.773112  1.007047
EAD ($MM)      0.693180  0.808313  1.074636  1.413704
M (Years)      5.000000  5.000000  4.703323  4.005414

Receiver Swap
                   0.02      0.04      0.06      0.08
EPE(1) ($MM)   0.478530  0.519562  0.800931  1.169682
EEPE(1) ($MM)  0.591714  0.566434  0.630899  0.715720
EAD ($MM)      0.814641  0.775295  0.866028  0.985160
M (Years)      5.000000  5.000000  5.000000  5.000000


In [198]:
airb_payer = []
airb_receiver = []
k_payer = []
k_receiver = []
rho_p = 0.24 - 0.12 * (1 - np.exp(-50 * probability_default))
temp_airb = norm.cdf((norm.ppf(probability_default) - np.sqrt(rho_p) * norm.ppf(0.001))/(np.sqrt(1 - rho_p))) - probability_default
for i in range(4):
    k_payer.append(k_calc(h_m_payer[i],probability_default))
    k_receiver.append(k_calc(h_m_receiver[i],probability_default))
    payer_temp = EAD_payer[i]*loss_given_default*(temp_airb)*k_calc(h_m_payer[i],probability_default)
    receiver_temp = EAD_receiver[i]*loss_given_default*(temp_airb)*k_calc(h_m_receiver[i],probability_default)
    airb_payer.append(payer_temp)
    airb_receiver.append(receiver_temp)
airb_capital=pd.DataFrame([airb_payer,k_payer,airb_receiver,k_receiver],columns=['0.02','0.04','0.06','0.08'],index=['AIRB_Payer ($MM)','k_Payer','AIRB_Receiver ($MM)','k_Receiver'])

In [199]:
print(airb_capital)

                         0.02      0.04      0.06      0.08
AIRB_Payer ($MM)     0.201588  0.235070  0.293776  0.328461
k_Payer              5.226487  5.226487  4.913012  4.175586
AIRB_Receiver ($MM)  0.236910  0.225468  0.251854  0.286500
k_Receiver           5.226487  5.226487  5.226487  5.226487


Q.5

In [278]:
T_sim=1
delta_t = 1/52
delta_T = 0.5
u = np.matrix([[1],
              [1]])
n_paths = 1000
hpar_list=[4.4836/100]
h_payer_EPE=[]
h_receiver_EPE=[]
h_payer_EEPE=[]
h_receiver_EEPE=[]
h_m_payer=[]
h_m_receiver=[]
lambda_f = 0.04

for hpar_value in hpar_list: # h_par loop for different values of fixed coupon
    total_payer_swap_array = np.zeros(521)
    total_receiver_swap_array = np.zeros(521)
    EE_star_payer=np.array([])
    EE_star_receiver=np.array([])
    cva_payer_final=np.array([])
    cva_receiver_final=np.array([])
    for j in range(n_paths): #n_path loop for monte carlo
        x_previous = np.matrix([[0],
                               [0]])
        x_previous_l = 0
        payer_swap_array = np.array([])
        receiver_swap_array = np.array([])
        z_array = np.random.normal(0,1,2*(int(T/delta_t))+1)
        lambda_t_path = np.array([])
        discount_factor_array = np.array([])
        lambda_factor_array = np.array([])
        cva_payer=np.array([])
        cva_receiver=np.array([])
        for i in range(0,int(T/delta_t)+1): # i loop for creating of weekly sample paths 
            t_iter = i*delta_t
            dx_t_l = (y_t_l_list[i] - k_l*x_previous_l)*delta_t + sigma_l*np.random.normal(0,1,1)*(delta_t**0.5)
            
            z_1 = z_array[i]
            z_2 = z_array[i+int(T/delta_t)]
            z_3 = rho_x * z_1 + ((1 - (rho_x**2))**0.5) * z_2
            z_t = np.matrix([[z_1],
                        [z_3]])
#             y_t = matrix_calc(t_iter)
            y_t=matrix_calc_list[i]
            P_0_t = np.exp(-a*t_iter-(b*(t_iter**2)/2))
            payer, receiver = payer_receiver(t_iter,x_previous,y_t,P_0_t,delta_T,hpar_value)
            dx_t = (y_t*u - kappa*x_previous)*delta_t + sigma_x.T*z_t*(delta_t**0.5)
            
            lambda_t_path = np.append(lambda_t_path, lambda_f + x_previous_l)
            
            discount_factor = np.exp(-(a* t_iter+(b*(t_iter**2)/2) + (x_previous[0,0] + x_previous[1,0])*t_iter))
            lambda_factor = np.exp(-(lambda_f+x_previous_l)*t_iter)
            
            discount_factor_array = np.append(discount_factor_array, discount_factor)
            lambda_factor_array = np.append(lambda_factor_array, lambda_factor)
            
            x_previous_l = x_previous_l + dx_t_l
            x_previous = x_previous + dx_t
            payer_swap_array = np.append(payer_swap_array,[max(receiver-payer,0)]) # array storing weekly exposure values of payer swap
            receiver_swap_array = np.append(receiver_swap_array,[max(payer-receiver,0)]) # # array storing weekly exposure values of receiver swap
        
        cva_payer = payer_swap_array * lambda_t_path * discount_factor_array * lambda_factor_array
        cva_receiver = receiver_swap_array * lambda_t_path * discount_factor_array * lambda_factor_array
        cva_payer_final = np.append(cva_payer_final, (1 - recovery_rate) * np.sum(cva_payer) * delta_t)
        cva_receiver_final = np.append(cva_receiver_final, (1 - recovery_rate) * np.sum(cva_receiver) * delta_t)
        #total_payer_swap_array = total_payer_swap_array + payer_swap_array
        #total_receiver_swap_array = total_receiver_swap_array + receiver_swap_array        
    
    cva_payer_final = np.sum(cva_payer_final)/n_paths # E(E(t)) array for payer swap 
    cva_receiver_final = np.sum(cva_receiver_final)/n_paths # E(E(t)) array for receiver swap

In [279]:
print('Payer_CVA:', cva_payer_final)
print('Receiver_CVA:', cva_receiver_final)

Payer_CVA: 0.5004825401344054
Receiver_CVA: 0.4501676093188898


The above figures are in MM$

Q.6

In [280]:
correlation_matrix = np.array([[1],[rho_x],[rho_l],
                             [rho_x],[1],[rho_Z_2],
                             [rho_l],[rho_Z_2],[1]])
correlation_matrix = correlation_matrix.reshape(3,3)

sigma_3_factor = np.matrix([[sigma_1,0,0],
                          [0,sigma_2,0],
                          [0,0,sigma_l]])

kappa_3_factor = np.matrix([[0,0,0],
                  [0,kappa_2,0],
                  [0,0,k_l]])

In [281]:
covariance_matrix = np.array([[1*sigma_1*sigma_1],[rho_x*sigma_1*sigma_2],[rho_l*sigma_1*sigma_l],
                             [rho_x*sigma_1*sigma_2],[1*sigma_2*sigma_2],[rho_Z_2*sigma_2*sigma_l],
                             [rho_l*sigma_1*sigma_l],[rho_Z_2*sigma_2*sigma_l],[1*sigma_l*sigma_l]])
covariance_matrix = covariance_matrix.reshape(3,3)

In [282]:
l_matrix = np.linalg.cholesky(correlation_matrix)

In [283]:
T_sim=1
delta_t = 1/52
delta_T = 0.5
u = np.matrix([[1],
              [1],
              [1]])
n_paths = 1000
hpar_list=[4.4836/100]
h_payer_EPE=[]
h_receiver_EPE=[]
h_payer_EEPE=[]
h_receiver_EEPE=[]
h_m_payer=[]
h_m_receiver=[]
lambda_f = 0.04

for hpar_value in hpar_list: # h_par loop for different values of fixed coupon
    total_payer_swap_array = np.zeros(521)
    total_receiver_swap_array = np.zeros(521)
    EE_star_payer=np.array([])
    EE_star_receiver=np.array([])
    cva_payer_final=np.zeros(521)
    cva_receiver_final=np.zeros(521)
    for j in range(n_paths): #n_path loop for monte carlo
        x_previous = np.matrix([[0],
                               [0],
                               [0]])
        #x_previous_l = 0
        payer_swap_array = np.array([])
        receiver_swap_array = np.array([])
        z_array = np.random.normal(0,1,3*(int(T/delta_t))+1)
        lambda_t_path = np.array([])
        discount_factor_array = np.array([])
        lambda_factor_array = np.array([])
        cva_payer=np.array([])
        cva_receiver=np.array([])
        for i in range(0,int(T/delta_t)+1): # i loop for creating of weekly sample paths 
            t_iter = i*delta_t
            #dx_t_l = (y_t_l_list[i] - k_l*x_previous_l)*delta_t + sigma_l*np.random.normal(0,1,1)*(delta_t**0.5)
            
            z_1 = z_array[i]
            z_2 = z_array[i+int(T/delta_t)]
            z_3 = z_array[i+int(T/delta_t)*2]
            z_t = np.matrix([[z_1],
                           [z_2],  
                        [z_3]])
#             y_t = matrix_calc(t_iter)
            y_t=matrix_calc_list[i]
            P_0_t = np.exp(-a*t_iter-(b*(t_iter**2)/2))
            payer, receiver = payer_receiver(t_iter,x_previous[0:2,:],y_t,P_0_t,delta_T,hpar_value)
            
            y_t_3_factor = np.matrix([[y_t.item(0),0,0],
                                  [0,y_t.item(3),0],
                                      [0,0,y_t_l_list[i]]])
            z_t = l_matrix * z_t
            dx_t = (y_t_3_factor*u - kappa_3_factor*x_previous)*delta_t + sigma_3_factor.T*z_t*(delta_t**0.5)
            
            lambda_t_path = np.append(lambda_t_path, lambda_f + x_previous[2,0])
            
            discount_factor = np.exp(-(a* t_iter+(b*(t_iter**2)/2) + (x_previous[0,0] + x_previous[1,0])*t_iter))
            lambda_factor = np.exp(-(lambda_f+x_previous[2,0])*t_iter)
            
            discount_factor_array = np.append(discount_factor_array, discount_factor)
            lambda_factor_array = np.append(lambda_factor_array, lambda_factor)
            #print(x_previous)
            x_previous = x_previous + dx_t
            payer_swap_array = np.append(payer_swap_array,[max(receiver-payer,0)]) # array storing weekly exposure values of payer swap
            receiver_swap_array = np.append(receiver_swap_array,[max(payer-receiver,0)]) # # array storing weekly exposure values of receiver swap
        
        cva_payer = lambda_t_path * payer_swap_array * discount_factor_array * lambda_factor_array
        cva_receiver = lambda_t_path * receiver_swap_array * discount_factor_array * lambda_factor_array
        cva_payer_final = np.append(cva_payer_final, (1 - recovery_rate) * np.sum(cva_payer) * delta_t)
        cva_receiver_final = np.append(cva_receiver_final, (1 - recovery_rate) * np.sum(cva_receiver) * delta_t)
        #total_payer_swap_array = total_payer_swap_array + payer_swap_array
        #total_receiver_swap_array = total_receiver_swap_array + receiver_swap_array        
    
    
    cva_payer_final = np.sum(cva_payer_final)/n_paths # E(E(t)) array for payer swap 
    cva_receiver_final = np.sum(cva_receiver_final)/n_paths # E(E(t)) array for receiver swap

In [284]:
print('Payer_CVA:', cva_payer_final)
print('Receiver_CVA:', cva_receiver_final)

Payer_CVA: 0.6661008583849382
Receiver_CVA: 0.2773234133128456


The above figures are in MM$

The wrong-way risk for 'A' can be the right-way risk for 'B', so it explains that for one of the party CVA decreases and it increases for the other. 

### Done